In [1]:
import sys
from os import path
import os
from scipy.spatial.distance import cdist
import numpy as np
import datetime
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy.stats import gaussian_kde
import matplotlib.lines as mlines
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib.offsetbox import AnchoredText
from datetime import datetime as dt
import utide

In [2]:
def datenumaz(d):
    return 366 + d.toordinal() + (d - dt.fromordinal(d.toordinal())).total_seconds()/(24*60*60)

In [3]:
basefol = '/Users/amrozeidan/Documents/hiwi/easygshpy/base'

In [4]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df_simul = pd.read_csv(path.join(basefol, 'telemac_variables','variables_all_stations' ,'free_surface_all_stations.dat' ) , 
                           header =0 , parse_dates = ['date'],date_parser = dateparse, index_col =0 , squeeze=True)
df_simul.set_index('date' , inplace = True)
df_simul

,Roscoff,StMalo,StHelierJersey,Cherbourg,LeHavre,Antifer,Dieppe,Boulogne,Calais,Dunkerque,...,Wierumergronden,Schiermannskoog,Lauwersoog,Huibertgat,BorkumS,BorkumF,Emshoern,Eemshaven,Dukegat,Delfzijl
date,,,,,,,,,,,,,,,,,,,,,
2015-01-06 00:00:00,-3.077342,-2.078520,-1.448774,0.915410,2.872966,3.022305,3.899223,3.453816,2.341209,1.395777,...,0.610120,1.156833,1.099325,0.967301,1.213985,1.330583,1.390983,1.407176,1.433399,1.444366
2015-01-06 00:10:00,-3.237792,-2.409397,-1.766929,0.738466,2.849072,3.023163,3.997797,3.614916,2.541929,1.671727,...,0.517727,1.101081,1.036836,0.886085,1.165504,1.301711,1.375816,1.393922,1.430602,1.454116
2015-01-06 00:20:00,-3.362437,-2.720419,-2.094769,0.589682,2.811419,3.009843,4.103075,3.737375,2.692562,1.905376,...,0.421623,1.037987,0.968140,0.795198,1.103759,1.260080,1.347809,1.368150,1.419434,1.458050
2015-01-06 00:30:00,-3.441948,-3.016208,-2.397147,0.437518,2.784743,2.970931,4.163237,3.788049,2.819094,2.110469,...,0.323882,0.970579,0.896396,0.696972,1.031731,1.205152,1.306874,1.329737,1.396626,1.456069
2015-01-06 00:40:00,-3.496505,-3.306890,-2.671501,0.239993,2.734252,2.911422,4.206853,3.845502,2.896434,2.260471,...,0.223830,0.898404,0.821673,0.591312,0.950239,1.138690,1.253037,1.277388,1.360903,1.447226
2015-01-06 00:50:00,-3.535882,-3.611383,-2.959901,0.043884,2.700744,2.845014,4.215614,3.895777,2.956033,2.378900,...,0.124166,0.823929,0.745704,0.480694,0.862152,1.062695,1.187716,1.212732,1.311972,1.427301
2015-01-06 01:00:00,-3.530883,-3.905649,-3.228913,-0.123798,2.625614,2.772796,4.147752,3.906502,3.002980,2.476306,...,0.023940,0.747972,0.667647,0.365730,0.768540,0.979803,1.113112,1.137913,1.250734,1.396029
2015-01-06 01:10:00,-3.500746,-4.185069,-3.475717,-0.284185,2.532588,2.650698,4.041566,3.899200,3.033301,2.540723,...,-0.074924,0.672164,0.588174,0.248752,0.667824,0.891269,1.030885,1.054448,1.179163,1.353500
2015-01-06 01:20:00,-3.452023,-4.417114,-3.691983,-0.488194,2.416065,2.511663,3.904344,3.854985,3.039341,2.624182,...,-0.172723,0.594997,0.506632,0.131695,0.561480,0.796787,0.942743,0.964587,1.100466,1.300998


In [5]:
dates = df_simul.index.tolist()
dates

[Timestamp('2015-01-06 00:00:00'),
 Timestamp('2015-01-06 00:10:00'),
 Timestamp('2015-01-06 00:20:00'),
 Timestamp('2015-01-06 00:30:00'),
 Timestamp('2015-01-06 00:40:00'),
 Timestamp('2015-01-06 00:50:00'),
 Timestamp('2015-01-06 01:00:00'),
 Timestamp('2015-01-06 01:10:00'),
 Timestamp('2015-01-06 01:20:00'),
 Timestamp('2015-01-06 01:30:00'),
 Timestamp('2015-01-06 01:40:00'),
 Timestamp('2015-01-06 01:50:00'),
 Timestamp('2015-01-06 02:00:00'),
 Timestamp('2015-01-06 02:10:00'),
 Timestamp('2015-01-06 02:20:00'),
 Timestamp('2015-01-06 02:30:00'),
 Timestamp('2015-01-06 02:40:00'),
 Timestamp('2015-01-06 02:50:00'),
 Timestamp('2015-01-06 03:00:00'),
 Timestamp('2015-01-06 03:10:00'),
 Timestamp('2015-01-06 03:20:00'),
 Timestamp('2015-01-06 03:30:00'),
 Timestamp('2015-01-06 03:40:00'),
 Timestamp('2015-01-06 03:50:00'),
 Timestamp('2015-01-06 04:00:00'),
 Timestamp('2015-01-06 04:10:00'),
 Timestamp('2015-01-06 04:20:00'),
 Timestamp('2015-01-06 04:30:00'),
 Timestamp('2015-01-

In [6]:
date_num = list(map(datenumaz,df_simul.index.tolist()))
date_num

[735970.0,
 735970.0069444445,
 735970.0138888889,
 735970.0208333334,
 735970.0277777778,
 735970.0347222222,
 735970.0416666666,
 735970.0486111111,
 735970.0555555555,
 735970.0625,
 735970.0694444445,
 735970.0763888889,
 735970.0833333334,
 735970.0902777778,
 735970.0972222222,
 735970.1041666666,
 735970.1111111111,
 735970.1180555555,
 735970.125,
 735970.1319444445,
 735970.1388888889,
 735970.1458333334,
 735970.1527777778,
 735970.1597222222,
 735970.1666666666,
 735970.1736111111,
 735970.1805555555,
 735970.1875,
 735970.1944444445,
 735970.2013888889,
 735970.2083333334,
 735970.2152777778,
 735970.2222222222,
 735970.2291666666,
 735970.2361111111,
 735970.2430555555,
 735970.25,
 735970.2569444445,
 735970.2638888889,
 735970.2708333334,
 735970.2777777778,
 735970.2847222222,
 735970.2916666666,
 735970.2986111111,
 735970.3055555555,
 735970.3125,
 735970.3194444445,
 735970.3263888889,
 735970.3333333334,
 735970.3402777778,
 735970.3472222222,
 735970.3541666666,
 7

In [7]:
elevations = df_simul['BorkumS'].to_numpy()
elevations

array([1.21398497, 1.16550434, 1.10375941, ..., 0.05524478, 0.16234304,
       0.26036274])

In [8]:
lib_func_fol = '/Users/amrozeidan/Downloads/pytides-master'

sys.path.append(path.join(lib_func_fol , 'pytides' ))
import pytides

In [12]:
from tide import Tide

demeaned = elevations

tide = Tide.decompose(demeaned, dates)

In [13]:
elevations - np.mean(elevations)

array([ 0.66841779,  0.61993716,  0.55819223, ..., -0.49032239,
       -0.38322414, -0.28520443])

In [14]:
import numpy as np
from pandas import DataFrame

constituent = [c.name for c in tide.model['constituent']]

df = pd.DataFrame(tide.model, index=constituent).drop('constituent', axis=1)

In [19]:
pTides = ['MM','MF','Q1','O1','K1','SO1','MU2','N2','NU2','M2','S2','2SM2','MO3','MN4','M4'
          ,'MS4','MK4','S4','M6','2MS6','S6','M8','M10','M12' ] 

In [20]:
pTides_for_pytides = ['Mm','Mf','Q1','O1','K1','SO1','mu2','N2','nu2','M2','S2','2SM2','MO3','MN4','M4'
          ,'MS4','MK4','S4','M6','2MS6','S6','M8','M10','M12' ] 

In [15]:
df

,amplitude,phase
Z0,0.545567,0.000000
M2,4811.703739,161.073436
S2,3218.643820,268.139415
N2,2639.839330,163.517563
K1,12641.318155,109.604261
M4,0.171678,23.266412
O1,7289.193336,318.707893
M6,0.018512,261.545943
MK3,0.191907,0.751446
S4,0.051839,172.522852


In [23]:
df.loc[pTides_for_pytides , ['amplitude' , 'phase']]

,amplitude,phase
Mm,NaN,NaN
Mf,NaN,NaN
Q1,23053.143278,128.366724
O1,7289.193336,318.707893
K1,12641.318155,109.604261
SO1,NaN,NaN
mu2,5245.311656,53.756405
N2,2639.839330,163.517563
nu2,1158.475379,34.599969
M2,4811.703739,161.073436


In [16]:
df_simul_matlab_a = pd.read_csv('/Users/amrozeidan/Downloads/EAZYgsh tools/testing_for_nico/base/coef_simulated/A_simul_all_stations.dat')
df_simul_matlab_a.set_index('tide' , inplace = True)
df_simul_matlab_a.head()

,Emshoern,Eemshaven,Dukegat,Delfzijl,BorkumS
tide,,,,,
MM,289.555357,292.882322,310.621050,334.931412,244.696079
NU2,191.368253,194.597297,199.142857,206.131317,168.011043
N2,181.375247,184.479048,188.825634,195.270159,159.121604
MF,42.312405,42.784532,45.332860,48.703205,35.877572
M2,23.012729,23.367411,23.926685,25.040884,20.260128


In [17]:
a = df_simul_matlab_a.loc[: , ['BorkumS']]
a.columns = ['a_matlab']

In [18]:
df_simul_matlab_g = pd.read_csv('/Users/amrozeidan/Downloads/EAZYgsh tools/testing_for_nico/base/coef_simulated/g_simul_all_stations.dat')
df_simul_matlab_g.set_index('tide' , inplace = True)
df_simul_matlab_g.head()

,Emshoern,Eemshaven,Dukegat,Delfzijl,BorkumS
tide,,,,,
MM,96.804670,96.812702,96.795143,96.787278,96.815861
NU2,275.304019,276.175337,281.189305,286.058272,262.336655
N2,188.269355,189.147377,194.194966,199.136112,175.251248
MF,51.924798,51.950114,51.930936,52.009440,51.879608
M2,5.236390,6.040082,10.897496,15.668607,352.443757


In [19]:
g = df_simul_matlab_g.loc[: , ['BorkumS']]
g.columns = ['g_matlab']

In [20]:
dfall = df.join(a, how = 'inner')
dfall = dfall.join(g , how = 'inner')
dfall = dfall.sort_index(axis = 1)

In [21]:
dfall

,a_matlab,amplitude,g_matlab,phase
M2,20.260128,4811.703739,352.443757,161.073436
S2,2.287778,3218.643820,274.415073,268.139415
N2,159.121604,2639.839330,175.251248,163.517563
K1,0.165136,12641.318155,343.022344,109.604261
M4,0.507177,0.171678,326.204804,23.266412
O1,0.611210,7289.193336,322.959148,318.707893
M6,0.023452,0.018512,349.278135,261.545943
S4,0.056551,0.051839,323.957250,172.522852
MN4,0.253678,0.123343,321.199294,18.144195
S6,0.011656,0.011799,45.839223,106.235054
